In [ ]:
# make all extras (nonlocal grid contributions) for spmv
# neighbours are grid blocks on another mpi rank
# command below
# cat ComputeSPMV_singleRank.ref ../python_scripts/temp_spmv_extra.cpp  >ComputeSPMV_ref.cpp && echo "return 0;}" >>ComputeSPMV_ref.cpp

In [ ]:
#def simplify_string # do this and redo debugging

In [1]:
def generate_offset(x_min, x_max, y_min, y_max, z_min, z_max):
    '''R(right)L are x , FB are y, UD are z
       generate dictionary of dictionaries
         first is label to denote which movements are present (RLFBUD, U, etc)
         second relates particular movement, within that set, to an offset 
         eg offset_dict['RFU']['001'] would be the offset for the cell in +ve z 
           direction in RFU corner of
           MPI cell (x and y) also available in this decomposition of global grid '''
    offset_dict = {}
    total='0'
    for ic in range(z_min,z_max+1):
        for ib in range(y_min,y_max+1):        
            for ia in range(x_min,x_max+1):
                if (ia==0 and ib==0 and ic==0):continue
                offset_dict["{}{}{}".format(ia,ib,ic)]="{}".format(total)
                if(abs(ia)==1 and abs(ib)==1 and abs(ic)==1):
                    total += '+1'
                if(abs(ia)==1 and abs(ib)==1 and ic==0):
                    total += '+nz'
                if(abs(ia)==0 and abs(ib)==1 and abs(ic)==1):
                    total += '+nx'
                if(abs(ia)==1 and abs(ib)==0 and abs(ic)==1):
                    total += '+ny'
                if(abs(ia)==1 and abs(ib)==0 and abs(ic)==0):
                    total += '+ny*nz'
                if(abs(ia)==0 and abs(ib)==1 and abs(ic)==0):
                    total += '+nx*nz'
                if(abs(ia)==0 and abs(ib)==0 and abs(ic)==1):
                    total += '+nx*ny'                    
    return offset_dict
#generate_offset(-1,1,-1,1,-1,1)

In [2]:
def corner_to_bulk(a,b,c, lower_a, upper_a, lower_b, upper_b, lower_c, upper_c, extra_offset):
    ''' a,b,c define corner - search further in this direction if bounds allow
     rename to corner_extend if a general sub can be written '''
    outstring = ""
    for ia in range(lower_a,upper_a + 1):
        for ib in range(lower_b,upper_b + 1):
            for ic in range(lower_c,upper_c + 1):
                #can't extend corner in opposite direction
                if ((ia == 0) and (ib==0) and (ic==0)):
                    continue
                #need at least 1 corner to match extension direction to add these
                if not ((a == ia) or (b == ib) or (c==ic)):
                    continue
#                if (abs(ia)==1 and abs(ib)==1 and abs(ic)==1):
#                    outstring += "-xv[nlocal+"+extra_offset[str(ia)+str(ib)+str(ic)]+"]\n"
                # 1 zero (faces)
                if(abs(ia)==1 and ib==0 and ic==0):
                     outstring+="".join(["-xv[nlocal+"+extra_offset[str(ia)+str(ib)+str(ic)]\
                      +"{}]\n".format(i) for i in general_indices_face('y','z',b,c)])
                if(ia==0 and abs(ib)==1 and ic==0):
                     outstring+="".join(["-xv[nlocal+"+extra_offset[str(ia)+str(ib)+str(ic)]\
                      +"{}]\n".format(i) for i in general_indices_face('x','z',a,c)])
                if(ia==0 and ib==0 and abs(ic)==1):
                     outstring+="".join(["-xv[nlocal+"+extra_offset[str(ia)+str(ib)+str(ic)]\
                      +"{}]\n".format(i) for i in general_indices_face('x','y',a,b)])

                #need 2 corners to match extension direction to add these
                if not (((a == ia)and(b == ib)) or ((a == ia)and(c==ic))\
                      or((b == ib)and(c == ic))):
                    continue                      
                if (abs(ia) ==0 and abs(ib)==1 and abs(ic)==1):# add 2
                     outstring+="".join(["-xv[nlocal+"+extra_offset[str(ia)+str(ib)+str(ic)]\
                      +"{}]\n".format(i) for i in general_indices_edge('x',a)])
                if (abs(ia) ==1 and abs(ib)==0 and abs(ic)==1):# add 2
                     outstring+="".join(["-xv[nlocal+"+extra_offset[str(ia)+str(ib)+str(ic)]\
                      +"{}]\n".format(i) for i in general_indices_edge('y',b)])
                if (abs(ia) ==1 and abs(ib)==1 and abs(ic)==0):# add 2
                     outstring+="".join(["-xv[nlocal+"+extra_offset[str(ia)+str(ib)+str(ic)]\
                      +"{}]\n".format(i) for i in general_indices_edge('z',c)])
                #need 3 corners to match extension direction to add these
                if ((a == ia)and(b == ib)and(c==ic)):
                     outstring+="-xv[nlocal+"+extra_offset[str(ia)+str(ib)+str(ic)]+"]\n"
    outstring+=";\n"
    return outstring

In [3]:
def zero_index_27pt_mpi(lower_x, upper_x, lower_y, upper_y, lower_z, upper_z, offset_dict):
    ''' corners '''
    i1,i2,i3 = 'x', 'y', 'z'
    outstring = ""

    def set_value(val, indx):
        if val == 0:
            return "i{} = 0;\n".format(indx)
        elif val == 1:
            return "i{} = n{}-1;\n".format(indx,indx)

    for ia in range(2):
        for ib in range(2):
            for ic in range(2):
                #if (ia, ib, )
                added_terms = corner_to_bulk(ia*2-1, ib*2-1, ic*2-1,
                             lower_x, upper_x, lower_y,upper_y, lower_z, upper_z, offset_dict)
                #
                if added_terms == ";\n":
                    continue
                outstring += set_value(ia, 'x')
                outstring += set_value(ib, 'y')
                outstring += set_value(ic, 'z')
                outstring += "yv[ix+iy*nx+iz*ny*nx] += \n"
                outstring += added_terms
    return outstring    

In [4]:
def general_indices_edge(axis, posn):
    ''' axis is string name, posn is -1 for start 1 for end '''
    if posn==-1:
        return("", "+1")
    elif posn==1:
        return("+n{}-2".format(axis), "+n{}-1".format(axis))
    else:
        assert(posn in [-1,1])

In [5]:
def general_indices_face(quick_axis, slow_axis, quick_posn, slow_posn):
    ''' alphabetically quick axis is lower '''
    assert(quick_axis < slow_axis)
    if (quick_posn == -1 and slow_posn == -1):
        return("", "+1", "+n{}".format(quick_axis), "+n{}+1".format(quick_axis))
    if (quick_posn == -1 and slow_posn == 1):
        return("+n{}*(n{}-2)".format  (quick_axis, slow_axis),
               "+n{}*(n{}-2)+1".format(quick_axis, slow_axis),
               "+n{}*(n{}-1)".format  (quick_axis, slow_axis),
               "+n{}*(n{}-1)+1".format(quick_axis, slow_axis))
    if (quick_posn == 1 and slow_posn == -1):
        return("+n{}-2".format  (quick_axis),
               "+n{}-1".format  (quick_axis),
               "+2*n{}-2".format(quick_axis),
               "+2*n{}-1".format(quick_axis))
    if (quick_posn == 1 and slow_posn == 1):
        return("+n{}*(n{}-1)-2".format(quick_axis, slow_axis),
               "+n{}*(n{}-1)-1".format(quick_axis, slow_axis),
               "+n{}*n{}-2".format    (quick_axis, slow_axis),
               "+n{}*n{}-1".format    (quick_axis, slow_axis))

In [6]:
def two_index_27pt_mpi(lower_x, upper_x, lower_y, upper_y, lower_z, upper_z, offset_dict):
    ''' faces - include more mpi ranks'''

    outstring = ""
    for a in [('x', lower_x, upper_x, offset_dict),
              ('y', lower_y, upper_y, offset_dict), 
              ('z', lower_z, upper_z, offset_dict) ]:

        outstring += face_to_bulk(*a)

    return outstring

In [7]:
def face_to_bulk(a, lower_a, upper_a, extra_offset):
    other_two_axes = {'x':('y','z'),'y':('x','z'),'z':('x','y')}
    outstring = ""

    for ia in range(lower_a,upper_a + 1):
        if (ia == 0):
            continue
        if (ia == 1):
            outstring += "i{}=n{}-1;\n".format(a,a)
        if (ia == -1):
            outstring += "i{}=0;\n".format(a)
        outstring+="for (i{}=1; i{}<n{}-1;i{}++)\n".format(other_two_axes[a][0],
                                                           other_two_axes[a][0],
                                                           other_two_axes[a][0],
                                                           other_two_axes[a][0])
        outstring+="{\n"
        outstring+="for (i{}=1; i{}<n{}-1;i{}++)\n".format(other_two_axes[a][1],
                                                           other_two_axes[a][1],
                                                           other_two_axes[a][1],
                                                           other_two_axes[a][1])
        # add 9 points
        outstring+="{\n yv[ix+iy*nx+iz*ny*nx] +=\n"
        if (a=='x'):
            outstring+="".join(["-xv[nlocal+"+extra_offset[str(ia)+str(0)+str(0)]\
               +"{}]\n".format(i) for i in expand_face_to_neighbour(*other_two_axes[a])])
        if (a=='y'):
            outstring+="".join(["-xv[nlocal+"+extra_offset[str(0)+str(ia)+str(0)]\
               +"{}]\n".format(i) for i in expand_face_to_neighbour(*other_two_axes[a])])
        if (a=='z'):
            outstring+="".join(["-xv[nlocal+"+extra_offset[str(0)+str(0)+str(ia)]\
               +"{}]\n".format(i) for i in expand_face_to_neighbour(*other_two_axes[a])])
        outstring+=";}\n"
        outstring+="}\n"

    return outstring

In [8]:
def expand_face_to_neighbour(quick_axis, slow_axis):
    ''' '''
    return("+i{}-1+(i{}-1)*n{}".format(quick_axis, slow_axis, quick_axis),
           "+i{}+(i{}-1)*n{}"  .format(quick_axis, slow_axis, quick_axis),
           "+i{}+1+(i{}-1)*n{}".format(quick_axis, slow_axis, quick_axis),
           "+i{}-1+(i{})*n{}"  .format(quick_axis, slow_axis, quick_axis),
           "+i{}+i{}*n{}"      .format(quick_axis, slow_axis, quick_axis),
           "+i{}+1+i{}*n{}"    .format(quick_axis, slow_axis, quick_axis),
           "+i{}-1+(i{}+1)*n{}".format(quick_axis, slow_axis, quick_axis),
           "+i{}+(i{}+1)*n{}"  .format(quick_axis, slow_axis, quick_axis),
           "+i{}+1+(i{}+1)*n{}".format(quick_axis, slow_axis, quick_axis))

In [9]:
def one_index_27pt_mpi(lower_x, upper_x, lower_y, upper_y, lower_z, upper_z, offset_dict):
    ''' edges - assume all mpi ranks'''

    outstring = ""
    for a in [('x', lower_y, upper_y, lower_z, upper_z, offset_dict),
              ('y', lower_x, upper_x, lower_z, upper_z, offset_dict), 
              ('z', lower_x, upper_x, lower_y, upper_y, offset_dict) ]:

        outstring += edge_to_bulk(*a)

    return outstring

In [10]:
def edge_to_bulk(a, lower_b, upper_b, lower_c, upper_c, extra_offset):
    ''' a, defines edge (x,y,z) - search further other directions if bounds allow '''
    other_two_axes = {'x':('y','z'),'y':('x','z'),'z':('x','y')}

    outstring = ""

    for ib in range(lower_b,upper_b + 1): #look for ways to expand in quicker direction
        for ic in range(lower_c,upper_c + 1):
            if(abs(ib)==1 and abs(ic)==0): #moving in quicker direction of b and c

                if a == 'x':
                    if(ib==-1): #y backwards
                        outstring+="i{}=0;\n".format(other_two_axes[a][0])#set y=0
                    if(ib==1): #y forwards
                        outstring+="i{}=n{}-1;\n".format(other_two_axes[a][0],other_two_axes[a][0])#set y=ny-1
                    outstring+="i{}=0;\n".format(other_two_axes[a][1])#z=0
                    outstring+="for (i{}=1; i{}<n{}-1;i{}++)\n".format(a,a,a,a)
                    outstring+="{\n yv[ix+iy*nx+iz*ny*nx] +=\n"
                    outstring+="".join(["-xv[nlocal+"+extra_offset[str(0)+str(ib)+str(ic)]\
                          +"{}]\n".format(i) for i in expand_edge_to_face('x','x','z',-1)])
                    outstring+=";}"
                    outstring+="i{}=n{}-1;\n".format(other_two_axes[a][1],other_two_axes[a][1])#z=nz-1
                    outstring+="for (i{}=1; i{}<n{}-1;i{}++)\n".format(a,a,a,a)
                    outstring+="{\n yv[ix+iy*nx+iz*ny*nx] +=\n"
                    outstring+="".join(["-xv[nlocal+"+extra_offset[str(0)+str(ib)+str(ic)]\
                          +"{}]\n".format(i) for i in expand_edge_to_face('x','x','z',1)])
                    outstring+=";}\n"

                if a == 'y':
                    if(ib==-1): #x backwards
                        outstring+="i{}=0;\n".format(other_two_axes[a][0])#set y=0
                    if(ib==1): #x forwards
                        outstring+="i{}=n{}-1;\n".format(other_two_axes[a][0],other_two_axes[a][0])#set y=ny-1
                    outstring+="i{}=0;\n".format(other_two_axes[a][1])
                    outstring+="for (i{}=1; i{}<n{}-1;i{}++)\n".format(a,a,a,a)
                    outstring+="{\n yv[ix+iy*nx+iz*ny*nx] +=\n"
                    outstring+="".join(["-xv[nlocal+"+extra_offset[str(ib)+str(0)+str(ic)]\
                          +"{}]\n".format(i) for i in expand_edge_to_face('y','y','z',-1)])
                    outstring+=";}"
                    outstring+="i{}=n{}-1;\n".format(other_two_axes[a][1],other_two_axes[a][1])#z=nz-1
                    outstring+="for (i{}=1; i{}<n{}-1;i{}++)\n".format(a,a,a,a)
                    outstring+="{\n yv[ix+iy*nx+iz*ny*nx] +=\n"
                    outstring+="".join(["-xv[nlocal+"+extra_offset[str(ib)+str(0)+str(ic)]\
                          +"{}]\n".format(i) for i in expand_edge_to_face('y','y','z',1)])
                    outstring+=";}\n"

                if a == 'z':
                    if(ib==-1): #y backwards
                        outstring+="i{}=0;\n".format(other_two_axes[a][0])#set y=0
                    if(ib==1): #y forwards
                        outstring+="i{}=n{}-1;\n".format(other_two_axes[a][0],other_two_axes[a][0])#set y=ny-1
                    outstring+="i{}=0;\n".format(other_two_axes[a][1])#z=0
                    outstring+="for (i{}=1; i{}<n{}-1;i{}++)\n".format(a,a,a,a)
                    outstring+="{\n yv[ix+iy*nx+iz*ny*nx] +=\n"
                    outstring+="".join(["-xv[nlocal+"+extra_offset[str(ib)+str(ic)+str(0)]\
                          +"{}]\n".format(i) for i in expand_edge_to_face('z','y','z',-1)])
                    outstring+=";}"
                    outstring+="i{}=n{}-1;\n".format(other_two_axes[a][1],other_two_axes[a][1])#z=nz-1
                    outstring+="for (i{}=1; i{}<n{}-1;i{}++)\n".format(a,a,a,a)
                    outstring+="{\n yv[ix+iy*nx+iz*ny*nx] +=\n"
                    outstring+="".join(["-xv[nlocal+"+extra_offset[str(ib)+str(ic)+str(0)]\
                          +"{}]\n".format(i) for i in expand_edge_to_face('z','y','z',1)])
                    outstring+=";}\n"

            if(abs(ib)==0 and abs(ic)==1): #moving in slower direction of b and c
                if a == 'x':
                    if(ic==-1): #z backwards
                        outstring+="i{}=0;\n".format(other_two_axes[a][1])#set y=0
                    if(ic==1): #z forwards
                        outstring+="i{}=n{}-1;\n".format(other_two_axes[a][1],other_two_axes[a][1])#set y=ny-1
                    outstring+="i{}=0;\n".format(other_two_axes[a][0])#y=0
                    outstring+="for (i{}=1; i{}<n{}-1;i{}++)\n".format(a,a,a,a)
                    outstring+="{\n yv[ix+iy*nx+iz*ny*nx] +=\n"
                    outstring+="".join(["-xv[nlocal+"+extra_offset[str(0)+str(ib)+str(ic)]\
                          +"{}]\n".format(i) for i in expand_edge_to_face('x','x','y',-1)])
                    outstring+=";}"
                    outstring+="i{}=n{}-1;\n".format(other_two_axes[a][0],other_two_axes[a][0])#y=nz-1
                    outstring+="for (i{}=1; i{}<n{}-1;i{}++)\n".format(a,a,a,a)
                    outstring+="{\n yv[ix+iy*nx+iz*ny*nx] +=\n"
                    outstring+="".join(["-xv[nlocal+"+extra_offset[str(0)+str(ib)+str(ic)]\
                          +"{}]\n".format(i) for i in expand_edge_to_face('x','x','y',1)])
                    outstring+=";}\n"

                if a == 'y':
                    if(ic==-1): #z backwards
                        outstring+="i{}=0;\n".format(other_two_axes[a][1])#set y=0
                    if(ic==1): #z forwards
                        outstring+="i{}=n{}-1;\n".format(other_two_axes[a][1],other_two_axes[a][1])#set y=ny-1
                    outstring+="i{}=0;\n".format(other_two_axes[a][0])#y=0
                    outstring+="for (i{}=1; i{}<n{}-1;i{}++)\n".format(a,a,a,a)
                    outstring+="{\n yv[ix+iy*nx+iz*ny*nx] +=\n"
                    outstring+="".join(["-xv[nlocal+"+extra_offset[str(ib)+str(0)+str(ic)]\
                          +"{}]\n".format(i) for i in expand_edge_to_face('y','x','y',-1)])
                    outstring+=";}"
                    outstring+="i{}=n{}-1;\n".format(other_two_axes[a][0],other_two_axes[a][0])#y=nz-1
                    outstring+="for (i{}=1; i{}<n{}-1;i{}++)\n".format(a,a,a,a)
                    outstring+="{\n yv[ix+iy*nx+iz*ny*nx] +=\n"
                    outstring+="".join(["-xv[nlocal+"+extra_offset[str(ib)+str(0)+str(ic)]\
                          +"{}]\n".format(i) for i in expand_edge_to_face('y','x','y',1)])
                    outstring+=";}\n"

                if a == 'z':
                    if(ic==-1): #z backwards
                        outstring+="i{}=0;\n".format(other_two_axes[a][1])#set y=0
                    if(ic==1): #z forwards
                        outstring+="i{}=n{}-1;\n".format(other_two_axes[a][1],other_two_axes[a][1])#set y=ny-1
                    outstring+="i{}=0;\n".format(other_two_axes[a][0])#y=0
                    outstring+="for (i{}=1; i{}<n{}-1;i{}++)\n".format(a,a,a,a)
                    outstring+="{\n yv[ix+iy*nx+iz*ny*nx] +=\n"
                    outstring+="".join(["-xv[nlocal+"+extra_offset[str(ib)+str(ic)+str(0)]\
                          +"{}]\n".format(i) for i in expand_edge_to_face('z','x','z',-1)])
                    outstring+=";}"
                    outstring+="i{}=n{}-1;\n".format(other_two_axes[a][0],other_two_axes[a][0])#y=nz-1
                    outstring+="for (i{}=1; i{}<n{}-1;i{}++)\n".format(a,a,a,a)
                    outstring+="{\n yv[ix+iy*nx+iz*ny*nx] +=\n"
                    outstring+="".join(["-xv[nlocal+"+extra_offset[str(ib)+str(ic)+str(0)]\
                          +"{}]\n".format(i) for i in expand_edge_to_face('z','x','z',1)])
                    outstring+=";}\n"
                  

            if(abs(ib)==1 and abs(ic)==1):
                if a == 'x':
                    if(ib==-1): #y backwards
                        outstring+="i{}=0;\n".format(other_two_axes[a][0])#set y=0
                    if(ib==1): #y forwards
                        outstring+="i{}=n{}-1;\n".format(other_two_axes[a][0],other_two_axes[a][0])#set y=ny-1
                    if(ic==-1): #z backwards
                        outstring+="i{}=0;\n".format(other_two_axes[a][1])#set z=0
                    if(ic==1): #z forwards
                        outstring+="i{}=n{}-1;\n".format(other_two_axes[a][1],other_two_axes[a][1])#set z=nz-1
                    outstring+="for (i{}=1; i{}<n{}-1;i{}++)\n".format(a,a,a,a)
                    outstring+="{\n yv[ix+iy*nx+iz*ny*nx] +=\n"
                    outstring+="".join(["-xv[nlocal+"+extra_offset[str(0)+str(ib)+str(ic)]\
                             +"{}]\n".format(i) for i in expand_edge_to_edge(a)])
                    outstring+=";}\n"
                if a == 'y':
                    if(ib==-1): 
                        outstring+="i{}=0;\n".format(other_two_axes[a][0])
                    if(ib==1): 
                        outstring+="i{}=n{}-1;\n".format(other_two_axes[a][0],other_two_axes[a][0])#set y=ny-1
                    if(ic==-1):
                        outstring+="i{}=0;\n".format(other_two_axes[a][1])
                    if(ic==1): 
                        outstring+="i{}=n{}-1;\n".format(other_two_axes[a][1],other_two_axes[a][1])#set z=nz-1
                    outstring+="for (i{}=1; i{}<n{}-1;i{}++)\n".format(a,a,a,a)
                    outstring+="{\n yv[ix+iy*nx+iz*ny*nx] +=\n"
                    outstring+="".join(["-xv[nlocal+"+extra_offset[str(ib)+str(0)+str(ic)]\
                             +"{}]\n".format(i) for i in expand_edge_to_edge(a)])
                    outstring+=";}\n"
                if a == 'z':
                    if(ib==-1): #y backwards
                        outstring+="i{}=0;\n".format(other_two_axes[a][0])#set y=0
                    if(ib==1): #y forwards
                        outstring+="i{}=n{}-1;\n".format(other_two_axes[a][0],other_two_axes[a][0])#set y=ny-1
                    if(ic==-1): #z backwards
                        outstring+="i{}=0;\n".format(other_two_axes[a][1])#set z=0
                    if(ic==1): #z forwards
                        outstring+="i{}=n{}-1;\n".format(other_two_axes[a][1],other_two_axes[a][1])#set z=nz-1
                    outstring+="for (i{}=1; i{}<n{}-1;i{}++)\n".format(a,a,a,a)
                    outstring+="{\n yv[ix+iy*nx+iz*ny*nx] +=\n"
                    outstring+="".join(["-xv[nlocal+"+extra_offset[str(ib)+str(ic)+str(0)]\
                             +"{}]\n".format(i) for i in expand_edge_to_edge(a)])
                    outstring+=";}\n"
                    
    outstring += ";\n"                
    return outstring

In [11]:
def expand_edge_to_edge(edge_axis):
    ''' Add 3 more points from edge '''
    return("+i{}-1".format(edge_axis),"+i{}".format(edge_axis),"+i{}+1".format(edge_axis))

In [15]:
def expand_edge_to_face(edge_axis, quick_axis, slow_axis, edge_posn):
    ''' alphabetically quick axis is lower 
        note face is made of edge and other ()'''
    assert(quick_axis < slow_axis)
#    outstring = "for( i{}=1; i{}< n{}-1; i{}++){\n".format(edge_axis, edge_axis, edge_axis) 
    #axis is i_edge*quick axis
    if (edge_axis == quick_axis and edge_posn == -1):
        return("+i{}-1"     .format(edge_axis), 
               "+i{}"       .format(edge_axis),
               "+i{}+1"     .format(edge_axis),
               "+n{}+i{}-1".format(quick_axis, edge_axis),
               "+n{}+i{}"  .format(quick_axis, edge_axis),
               "+n{}+i{}+1".format(quick_axis, edge_axis))
    if (edge_axis == quick_axis and edge_posn == 1):
#    if (edge_axis == quick_axis and slow_posn == 1):
        return("+i{}+(n{}-2)*n{}-1".format(edge_axis, slow_axis, quick_axis), 
               "+i{}+(n{}-2)*n{}"  .format(edge_axis, slow_axis, quick_axis),
               "+i{}+(n{}-2)*n{}+1".format(edge_axis, slow_axis, quick_axis),
               "+i{}+(n{}-1)*n{}-1".format(edge_axis, slow_axis, quick_axis),
               "+i{}+(n{}-1)*n{}"  .format(edge_axis, slow_axis, quick_axis),
               "+i{}+(n{}-1)*n{}+1".format(edge_axis, slow_axis, quick_axis))
#    if (edge_axis == slow_axis and quick_posn == -1):
    if (edge_axis == slow_axis and edge_posn == -1):
        return("+(i{}-1)*n{}"  .format(edge_axis, quick_axis),
               "+(i{}-1)*n{}+1".format(edge_axis, quick_axis),
               "+(i{})*n{}"    .format(edge_axis, quick_axis),
               "+(i{})*n{}+1".format(edge_axis, quick_axis),
               "+(i{}+1)*n{}"  .format(edge_axis, quick_axis),
               "+(i{}+1)*n{}+1".format(edge_axis, quick_axis))
#    if (edge_axis == slow_axis and quick_posn == 1):
    if (edge_axis == slow_axis and edge_posn == 1):
        return("+(i{}-1)*n{}+n{}-2".format(edge_axis, quick_axis, quick_axis),
               "+(i{}-1)*n{}+n{}-1".format(edge_axis, quick_axis, quick_axis),
               "+(i{})*n{}+n{}-2".format(edge_axis, quick_axis, quick_axis),
               "+(i{})*n{}+n{}-1".format(edge_axis, quick_axis, quick_axis),
               "+(i{}+1)*n{}+n{}-2".format(edge_axis, quick_axis, quick_axis),
               "+(i{}+1)*n{}+n{}-1".format(edge_axis, quick_axis, quick_axis))

In [16]:
def print_extra_MPI_contributions():
    outstring=""
    for xl in [['if(ipx > 0)',-1], ['else',0]]:
        outstring+=xl[0] + "\n"
        outstring+='{\n'
        for xh in [['if(ipx < npx - 1)',1], ['else',0]]:
            outstring+="  " + xh[0] + "\n"
            outstring+='{\n'
            for yl in [['if(ipy > 0)',-1], ['else',0]]:
                outstring+="    " + yl[0] + "\n"
                outstring+='{\n'
                for yh in [['if(ipy < npy - 1)',1], ['else',0]]:
                    outstring+="      " + yh[0] + "\n"
                    outstring+='{\n'
                    for zl in [['if(ipz > 0)',-1], ['else',0]]:
                        outstring+="        " + zl[0] + "\n"
                        outstring+='{\n'
                        for zh in [['if(ipz < npz - 1)',1], ['else',0]]:
                            outstring+="            "+ zh[0] + "\n"
                            outstring+='{\n'
                            bounds = (xl[1],xh[1],yl[1],yh[1],zl[1],zh[1])
                            offset_dict = generate_offset(*bounds)
#                            outstring+='std::cout <<A.geom->rank<<" "<<'+str(xl[1])+'<<" "<<'+str(xh[1])+'<<" "<<'+\
#                            str(yl[1])+'<<" "<<'+str(yh[1])+'<<" "<<'+str(zl[1])+'<<" "<<'+str(zh[1])+'<<std::endl;'
                            outstring += zero_index_27pt_mpi(*bounds, offset_dict)
                            outstring += one_index_27pt_mpi (*bounds, offset_dict)
                            #faces
                            outstring += two_index_27pt_mpi (*bounds, offset_dict)

                            outstring+="}//ipz < npz - 1\n"
                        outstring+='          }//ipz > 0 \n'
                    outstring+='        }//ipy < npy - 1 \n'
                outstring+='      }//ipy > 0 \n'
            outstring+='    }//ipx < npx - 1 \n'
        outstring+=' }//ipx > 0 \n'
    #outstring+='}//x1 \n'
    return outstring
#print(print_if_statements())#.count("BOUNDS"))
with open('temp_spmv_extra.cpp', 'w') as outfile:
    outfile.write(print_extra_MPI_contributions())

In [181]:
def temp_string():
    bounds = (-1, 0,-1, 1, 0, 1)
    offset_dict = generate_offset(*bounds)
    outstring=""#'std::cout <<A.geom->rank<<" "<<'+str(xl[1])+'<<" "<<'+str(xh[1])+'<<" "<<'+\
    #str(yl[1])+'<<" "<<'+str(yh[1])+'<<" "<<'+str(zl[1])+'<<" "<<'+str(zh[1])+'<<std::endl;'
    #print(bounds)
    #corners
    #try:
    outstring += zero_index_27pt_mpi(*bounds, offset_dict)
    return outstring
print( temp_string().replace('for', '#for')\
      .replace('-xv[','print(').replace(']',')').replace('yv', '#yv') )

ix = 0;
iy = 0;
iz = 0;
#yv[ix+iy*nx+iz*ny*nx) += 
print(nlocal+0)
print(nlocal+0+1)
print(nlocal+0+nz+nx*nz)
print(nlocal+0+nz+nx*nz+1)
print(nlocal+0+nz+nx*nz+ny)
print(nlocal+0+nz+nx*nz+ny+1)
print(nlocal+0+nz)
print(nlocal+0+nz+1)
print(nlocal+0+nz+nx)
print(nlocal+0+nz+nx+1)
;
ix = 0;
iy = 0;
iz = nz-1;
#yv[ix+iy*nx+iz*ny*nx) += 
print(nlocal+0+nz-2)
print(nlocal+0+nz-1)
print(nlocal+0+nz+nx*nz+ny*(nz-2))
print(nlocal+0+nz+nx*nz+ny*(nz-2)+1)
print(nlocal+0+nz+nx*nz+ny*(nz-1))
print(nlocal+0+nz+nx*nz+ny*(nz-1)+1)
print(nlocal+0+nz+nx*nz+ny*nz+nz+nx*nz+1+nx)
print(nlocal+0+nz+nx*nz+ny*nz+nz+nx*nz+1+nx+1)
print(nlocal+0+nz+nx*(nz-2))
print(nlocal+0+nz+nx*(nz-2)+1)
print(nlocal+0+nz+nx*(nz-1))
print(nlocal+0+nz+nx*(nz-1)+1)
print(nlocal+0+nz+nx*nz+ny*nz+nz+nx*nz+1)
print(nlocal+0+nz+nx*nz+ny*nz+nz+nx*nz+1+1)
print(nlocal+0+nz+nx*nz+ny*nz+nz+nx*nz+1+nx+ny)
print(nlocal+0+nz+nx*nz+ny*nz+nz+nx*nz+1+nx+ny+1)
print(nlocal+0+nz+nx*nz+ny*nz+nz+nx*nz+1+nx+ny+nx)
print(nlocal+0+nz+nx*nz+ny*nz+

In [176]:
def temp_eval():
    nlocal = 64
    nx,ny,nz = 4,4,4
    #ix,iy,iz = 2, 0 , 1
    print(nlocal+0)
    print(nlocal+0+1)
    print(nlocal+0+nz+nx*nz)
    print(nlocal+0+nz+nx*nz+1)
    print(nlocal+0+nz+nx*nz+ny)
    print(nlocal+0+nz+nx*nz+ny+1)
    print(nlocal+0+nz)
    print(nlocal+0+nz+1)
    print(nlocal+0+nz+nx)
    print(nlocal+0+nz+nx+1)
temp_eval()

64
65
84
85
88
89
68
69
72
73


In [140]:
import math

In [137]:
def col_to_gridpt_local(col):
    nx,ny,nz=4,4,4
    x = col%nx
    y = (math.floor(col/nx))%ny
    z = (math.floor(col/(nx*ny)))%nz
    return (x,y,z)

In [183]:
[col_to_gridpt_local(i) for i in [48,60]]

[(0, 0, 3), (0, 3, 3)]

In [178]:
#[col_to_gridpt_local(i) for i in [3,12,15,48,51,60]]